In [1]:
import os
import re

import numpy as np
import pandas as pd

# os.chdir("/home/yujia/Project/2023-07-20-individual_MR/dat/02_base_data/")

In [2]:
CAD = pd.read_csv("/mnt/md0/yujia/project/2023-07-20-individual_MR/dat/02_base_data/CAD/download/CLEAN.CAD.sumstats.rsids.120221.tsv.gz", sep="\t", compression = 'gzip')

print("Total SNP in the GWAS data: ")
print(CAD.shape)

Total SNP in the GWAS data: 
(5828377, 22)


In [3]:
CAD

,CHR,BP,SNP,A1,A2,N,beta,SE,P,EAF,...,N.Cases,N.Controls,homN_Allele2_cases,hetN_Allele2_cases,homN_Allele2_ctrls,hetN_Allele2_ctrls,OR,imputationInfo,EAC,SNP.chrps
0,1,832359.0,rs9697294,C,T,22008,-0.514566,0.243603,0.034659,0.011590,...,1110,20898,1,13,5,453,0.597760,0.853902,510.141176,1:832359:C:T
1,1,834056.0,rs28482280,A,C,22008,-0.480629,0.242051,0.047072,0.014507,...,1110,20898,1,13,5,457,0.618394,0.706821,638.549020,1:834056:A:C
2,1,834263.0,rs143031236,C,T,22008,-0.517446,0.247901,0.036860,0.010260,...,1110,20898,1,13,5,450,0.596041,0.916905,451.588235,1:834263:C:T
3,1,838665.0,rs28678693,T,C,22008,-0.498936,0.233908,0.032921,0.011059,...,1110,20898,1,13,5,460,0.607176,0.983308,486.788235,1:838665:T:C
4,1,838732.0,rs28477624,G,A,22008,-0.512187,0.242385,0.034591,0.010589,...,1110,20898,1,13,5,459,0.599184,0.952235,466.101961,1:838732:G:A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5828372,22,51164115.0,rs5770996,C,T,22008,0.039691,0.051607,0.441827,0.471853,...,1110,20898,276,523,4930,10192,1.040490,0.876210,20769.070588,22:51164115:C:T
5828373,22,51164287.0,rs6009957,T,C,22008,0.025579,0.052435,0.625681,0.362718,...,1110,20898,161,502,2964,9555,1.025908,0.932783,15965.388235,22:51164287:T:C
5828374,22,51165664.0,rs8137951,G,A,22008,0.019674,0.052754,0.709193,0.357103,...,1110,20898,157,501,2918,9534,1.019869,0.928185,15718.266667,22:51165664:G:A
5828375,22,51174939.0,rs73174435,C,T,22008,-0.097370,0.227792,0.669050,0.011928,...,1110,20898,2,20,8,489,0.907220,0.985402,525.035294,22:51174939:C:T


In [4]:
# rename the data
CAD.rename(columns={"CHR":"CHR", "BP":"BP", "SNP":"SNP", "A1":"A1", "A2":"A2", "EAF":"EAF", "beta":"BETA", "N":"N", "SE":"SE", "P":"P"}, inplace=True)

# convert EAF to MAF
CAD['MAF'] = CAD['EAF'].apply(lambda x : x if x <= 0.5 else 1-x)

# Uppercase the A1 and A2
CAD['A1'] = CAD['A1'].str.upper()
CAD['A2'] = CAD['A2'].str.upper()

In [5]:
CAD

,CHR,BP,SNP,A1,A2,N,BETA,SE,P,EAF,...,N.Controls,homN_Allele2_cases,hetN_Allele2_cases,homN_Allele2_ctrls,hetN_Allele2_ctrls,OR,imputationInfo,EAC,SNP.chrps,MAF
0,1,832359.0,rs9697294,C,T,22008,-0.514566,0.243603,0.034659,0.011590,...,20898,1,13,5,453,0.597760,0.853902,510.141176,1:832359:C:T,0.011590
1,1,834056.0,rs28482280,A,C,22008,-0.480629,0.242051,0.047072,0.014507,...,20898,1,13,5,457,0.618394,0.706821,638.549020,1:834056:A:C,0.014507
2,1,834263.0,rs143031236,C,T,22008,-0.517446,0.247901,0.036860,0.010260,...,20898,1,13,5,450,0.596041,0.916905,451.588235,1:834263:C:T,0.010260
3,1,838665.0,rs28678693,T,C,22008,-0.498936,0.233908,0.032921,0.011059,...,20898,1,13,5,460,0.607176,0.983308,486.788235,1:838665:T:C,0.011059
4,1,838732.0,rs28477624,G,A,22008,-0.512187,0.242385,0.034591,0.010589,...,20898,1,13,5,459,0.599184,0.952235,466.101961,1:838732:G:A,0.010589
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5828372,22,51164115.0,rs5770996,C,T,22008,0.039691,0.051607,0.441827,0.471853,...,20898,276,523,4930,10192,1.040490,0.876210,20769.070588,22:51164115:C:T,0.471853
5828373,22,51164287.0,rs6009957,T,C,22008,0.025579,0.052435,0.625681,0.362718,...,20898,161,502,2964,9555,1.025908,0.932783,15965.388235,22:51164287:T:C,0.362718
5828374,22,51165664.0,rs8137951,G,A,22008,0.019674,0.052754,0.709193,0.357103,...,20898,157,501,2918,9534,1.019869,0.928185,15718.266667,22:51165664:G:A,0.357103
5828375,22,51174939.0,rs73174435,C,T,22008,-0.097370,0.227792,0.669050,0.011928,...,20898,2,20,8,489,0.907220,0.985402,525.035294,22:51174939:C:T,0.011928


In [6]:
# reorder the column
cols = ["CHR", "BP", "SNP", "A1", "A2", "N", "SE", "P", "BETA", "MAF", "EAF"]
CAD = CAD[cols]

# Remove NaN row
CAD = CAD.dropna()

# convert to correct column type
CAD = CAD.astype({'CHR':'int', 'BP':'int', "SNP":'str', 'A1':'str', 'A2':'str', 'N':'int', 'SE':'float', 'P':'float', 'BETA':'float', 'MAF':'float', 'EAF':'float'})

# remove SNP without SNPID
CAD = CAD[CAD['SNP'].str.startswith('rs')]

# save the formatted results
CAD.to_csv("/mnt/md0/yujia/project/2023-07-20-individual_MR/dat/02_base_data/CAD/QC_SAS/CAD_formated_nadrop.tsv.gz", sep="\t", index=False)

print("SNP left after basic QC: ")
print(CAD.shape)

SNP left after basic QC: 
(5719364, 11)


In [7]:
CAD

,CHR,BP,SNP,A1,A2,N,SE,P,BETA,MAF,EAF
0,1,832359,rs9697294,C,T,22008,0.243603,0.034659,-0.514566,0.011590,0.011590
1,1,834056,rs28482280,A,C,22008,0.242051,0.047072,-0.480629,0.014507,0.014507
2,1,834263,rs143031236,C,T,22008,0.247901,0.036860,-0.517446,0.010260,0.010260
3,1,838665,rs28678693,T,C,22008,0.233908,0.032921,-0.498936,0.011059,0.011059
4,1,838732,rs28477624,G,A,22008,0.242385,0.034591,-0.512187,0.010589,0.010589
...,...,...,...,...,...,...,...,...,...,...,...
5828372,22,51164115,rs5770996,C,T,22008,0.051607,0.441827,0.039691,0.471853,0.471853
5828373,22,51164287,rs6009957,T,C,22008,0.052435,0.625681,0.025579,0.362718,0.362718
5828374,22,51165664,rs8137951,G,A,22008,0.052754,0.709193,0.019674,0.357103,0.357103
5828375,22,51174939,rs73174435,C,T,22008,0.227792,0.669050,-0.097370,0.011928,0.011928


In [8]:
# =============================
# Step 2: remove SNPs with MAF < 0.01
# =============================

cmd = "gunzip -c /mnt/md0/yujia/project/2023-07-20-individual_MR/dat/02_base_data/CAD/QC_SAS/CAD_formated_nadrop.tsv.gz |" + \
      "awk 'NR==1 || ($10 > 0.01) {print}' |" + \
      "gzip > /mnt/md0/yujia/project/2023-07-20-individual_MR/dat/02_base_data/CAD/QC_SAS/CAD.standardGWASQC.gz"
os.system(cmd)

cmd = "zcat /mnt/md0/yujia/project/2023-07-20-individual_MR/dat/02_base_data/CAD/QC_SAS/CAD.standardGWASQC.gz | wc -l"
print("SNP left after removing SNPs with MAF < 0.01: ")
print(os.popen(cmd).read())

SNP left after removing SNPs with MAF < 0.01: 
5502659



In [9]:
# =============================
# Step 3: remove duplicate SNPS
# =============================

cmd = "gunzip -c /mnt/md0/yujia/project/2023-07-20-individual_MR/dat/02_base_data/CAD/QC_SAS/CAD.standardGWASQC.gz |" + \
      "awk '{seen[$3]++; if(seen[$3]==1){ print}}' |" + \
      "gzip - > /mnt/md0/yujia/project/2023-07-20-individual_MR/dat/02_base_data/CAD/QC_SAS/CAD.nodup.gz"
os.system(cmd)

cmd = "zcat /mnt/md0/yujia/project/2023-07-20-individual_MR/dat/02_base_data/CAD/QC_SAS/CAD.nodup.gz | wc -l"
print("SNP left after removing duplicate SNPs: ")
print(os.popen(cmd).read())

SNP left after removing duplicate SNPs: 
5502659



In [10]:
# =============================
# Step 4: keep non-ambiguous SNPS
# =============================

cmd = "gunzip -c /mnt/md0/yujia/project/2023-07-20-individual_MR/dat/02_base_data/CAD/QC_SAS/CAD.nodup.gz | " + \
      "awk '!( ($4==\"A\" && $5==\"T\") || ($4==\"T\" && $5==\"A\") || ($4==\"G\" && $5==\"C\") || ($4==\"C\" && $5==\"G\")) {print}' | " + \
      "gzip > /mnt/md0/yujia/project/2023-07-20-individual_MR/dat/02_base_data/CAD/QC_SAS/CAD.QC.gz"
os.system(cmd)

cmd = "zcat /mnt/md0/yujia/project/2023-07-20-individual_MR/dat/02_base_data/CAD/QC_SAS/CAD.QC.gz | wc -l"
print("SNP left after removing ambiguous SNPs: ")
print(os.popen(cmd).read())

SNP left after removing ambiguous SNPs: 
4690726

